## Imports

In [1]:
from geopy.geocoders import GoogleV3
import numpy as np
import geocoder
import json
import pandas as pd
from geojson import Feature, FeatureCollection, Point
import tabula
from tabula.io import read_pdf
import re
import os

## Import data from PDF

In [2]:
# Gotta figure out how to get rid of the java error.
# SO: https://stackoverflow.com/questions/54817211/java-command-is-not-found-from-this-python-process-please-ensure-java-is-inst

dsf = tabula.read_pdf('condo.pdf', pages='all',lattice=True)

## Pandas Manipulation

In [3]:
df = dsf[0]
df.columns = df.iloc[0]
df = df.drop(df.index[0])
### This next line gets ride of the Nan ###
df = df.iloc[: , 1:]
df = df.replace(np.nan, 'Not Available', regex=True)
df = df.rename(columns={'Building / Address / City': 'building_address_city','Days on\rMarket':'days_on_market',\
                        'Price /\rSq. Ft.':'price_per_sqft'})
df['geo_address'] = df['building_address_city']
df['geo_address'] = df['geo_address'].map(lambda x: re.sub(r'\r', ' ', x))
df = df.replace('\n',' ',regex=True)
df2 = df['geo_address'].str.extract(r'(?P<building_name>\D*)(?P<address_only>\d.*)', expand=True)

Review this later (it's about how to properly merge DFs): https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

In [4]:
result = pd.merge(df,df2,left_index=True, right_index=True)
df = result

In [5]:
df

,building_address_city,List Date,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,Buyer Broker,geo_address,building_name,address_only
1,Edition Residences\r2901 Collins Ave 901\rMia...,7/15/2022,8/10/2022,3,"$7,200,000.00",2756,"$2,612.48",Marlene Goldman,Beachfront Realty Inc,Roland Ortiz,One Sotheby's INT'L Realty,Edition Residences 2901 Collins Ave 901 Miami...,Edition Residences,2901 Collins Ave 901 Miami Beach
2,Murano at Portofino\r1000 S Pointe Dr 301\rMia...,5/18/2022,8/10/2022,41,"$6,300,000.00",3365,"$1,872.21",Dina Goldentayer,Douglas Elliman,Denise Gomm Santos PA,Compass Florida LLC,Murano at Portofino 1000 S Pointe Dr 301 Miami...,Murano at Portofino,1000 S Pointe Dr 301 Miami Beach
3,Park Grove\r2821 S Bayshore Dr 12C\rCoconut Grove,6/14/2022,8/8/2022,9,"$3,550,000.00",1954,"$1,816.79",Jo-Ann Forster,Compass Florida LLC,David Siddons,Douglas Elliman,Park Grove 2821 S Bayshore Dr 12C Coconut Grove,Park Grove,2821 S Bayshore Dr 12C Coconut Grove
4,Saint Regis\r9705 Collins Ave 1404N\rBal Harbour,4/28/2022,8/10/2022,90,"$3,250,000.00",1685,"$1,928.78",Jose Correa Lima,Yaffe International Realty,Sergio Balinotti,"BRG International , LLC",Saint Regis 9705 Collins Ave 1404N Bal Harbour,Saint Regis,9705 Collins Ave 1404N Bal Harbour
5,Majestic Tower\r9601 Collins Ave 1703\rBal Har...,4/19/2022,8/9/2022,50,"$2,800,000.00",2160,"$1,296.30",Sergio Balinotti,"BRG International , LLC",Matias Alem,"BRG International , LLC",Majestic Tower 9601 Collins Ave 1703 Bal Harbour,Majestic Tower,9601 Collins Ave 1703 Bal Harbour
6,Ocean Club\r711 Crandon Blvd 202\rKey Biscayne,7/22/2022,8/10/2022,1,"$2,620,000.00",2080,"$1,259.62",Geysa Guarconi,KB Realty,Geysa Guarconi,KB Realty,Ocean Club 711 Crandon Blvd 202 Key Biscayne,Ocean Club,711 Crandon Blvd 202 Key Biscayne
7,The Bond\r1080 Brickell Ave 3500-3501\rMiami,10/12/2021,8/8/2022,303,"$1,910,000.00",3051,$700.40,Lida Juliao,Cervera Real Estate Inc.,Robert Himelfarb,Global Luxury Realty LLC,The Bond 1080 Brickell Ave 3500-3501 Miami,The Bond,1080 Brickell Ave 3500-3501 Miami
8,Blue Diamond Condo\r4779 Collins Ave 4002\rMia...,6/12/2022,8/10/2022,25,"$1,810,000.00",1530,"$1,183.01",Claudia Whitechurch,Coldwell Banker Realty,Seth Cohen,Brown Harris Stevens,Blue Diamond Condo 4779 Collins Ave 4002 Miami...,Blue Diamond Condo,4779 Collins Ave 4002 Miami Beach
9,Solimar\r9559 Collins Ave S7-H\rSurfside,7/11/2022,8/12/2022,8,"$1,800,000.00",1980,$909.09,Edward Ohayon,Jonathan Solomon Realty,Shneur Shapira,The Brokerage S Fl Real Estate,Solimar 9559 Collins Ave S7-H Surfside,Solimar,9559 Collins Ave S7-H Surfside
10,T0rump Royale\r18201 Collins Ave 3809\rSunny I...,3/4/2022,8/12/2022,108,"$1,775,000.00",1722,"$1,030.78",Yanina Velez,"Grand Realty of America, Corp.",Milana Nosikovsky PA,Exclusively Baronoff Rlty LLC,T0rump Royale 18201 Collins Ave 3809 Sunny Isl...,T,0rump Royale 18201 Collins Ave 3809 Sunny Isle...


In [6]:
df.columns

Index(['building_address_city', 'List Date', 'Close Date', 'days_on_market',
       'Sale Price', 'Sq. Ft.', 'price_per_sqft', 'Agent', 'Listing Broker',
       'Buyer Agent', 'Buyer Broker', 'geo_address', 'building_name',
       'address_only'],
      dtype='object')

## Geocoder Setup and Execution

In [7]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [8]:
df['loc'] = df['address_only'].apply(geolocator.geocode, timeout=10)

In [9]:
df['loc']

1     (2901 Collins Ave #901, Miami Beach, FL 33140,...
2     (1000 South Pointe Dr APT 301, Miami Beach, FL...
3     (2821 S Bayshore Dr #12c, Miami, FL 33133, USA...
4     (9705 Collins Ave #1404n, Bal Harbour, FL 3315...
5     (9601 Collins Ave APT 1703, Bal Harbour, FL 33...
6     (711 Crandon Blvd APT 202, Key Biscayne, FL 33...
7     (1080 Brickell Ave 3500 3501, Miami, FL 33131,...
8     (Blue Diamond Tower, 4779 Collins Ave APT 4002...
9     (9559 Collins Ave s7 h, Surfside, FL 33154, US...
10    (Trump Royale, 18201 Collins Ave #3809, Sunny ...
Name: loc, dtype: object

In [10]:
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)

Columns must be same length as key explained
- https://cumsum.wordpress.com/2021/06/01/pandas-valueerror-columns-must-be-same-length-as-key/

Creating `df2` based on `df` row indicies.

https://datascience.stackexchange.com/questions/77033/creating-a-new-dataframe-with-specific-row-numbers-from-another

What's tripping this error is when the lat/lon/alititude generator is fed a Nan value to parse - it's effectively trying to split a `null` value into three separate parts - which is impossible.

The code below effectively caputres the anatomy of this error. It creates a new `df` based on a list of indicies from an existing one. Row `56` has no `point` since the `geocoder` couldn't find coordinates for it. The other three do. Splitting the `point` column fails only when row `56` - with `null` values - is attempted.
```
df2 = df.iloc[[56,57,58,59],:]
df2[['lat','lon','altitude']] = pd.DataFrame(df2['point'].to_list(),index=df2.index)
```

In [15]:
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

df

,building_address_city,List Date,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,Buyer Broker,geo_address,building_name,address_only,loc,point,lat,lon,altitude
1,Edition Residences\r2901 Collins Ave 901\rMia...,7/15/2022,8/10/2022,3,"$7,200,000.00",2756,"$2,612.48",Marlene Goldman,Beachfront Realty Inc,Roland Ortiz,One Sotheby's INT'L Realty,Edition Residences 2901 Collins Ave 901 Miami...,Edition Residences,2901 Collins Ave 901 Miami Beach,"(2901 Collins Ave #901, Miami Beach, FL 33140,...","(25.8051273, -80.1241113, 0.0)",25.805127,-80.124111,0.0
2,Murano at Portofino\r1000 S Pointe Dr 301\rMia...,5/18/2022,8/10/2022,41,"$6,300,000.00",3365,"$1,872.21",Dina Goldentayer,Douglas Elliman,Denise Gomm Santos PA,Compass Florida LLC,Murano at Portofino 1000 S Pointe Dr 301 Miami...,Murano at Portofino,1000 S Pointe Dr 301 Miami Beach,"(1000 South Pointe Dr APT 301, Miami Beach, FL...","(25.7690044, -80.1381107, 0.0)",25.769004,-80.138111,0.0
3,Park Grove\r2821 S Bayshore Dr 12C\rCoconut Grove,6/14/2022,8/8/2022,9,"$3,550,000.00",1954,"$1,816.79",Jo-Ann Forster,Compass Florida LLC,David Siddons,Douglas Elliman,Park Grove 2821 S Bayshore Dr 12C Coconut Grove,Park Grove,2821 S Bayshore Dr 12C Coconut Grove,"(2821 S Bayshore Dr #12c, Miami, FL 33133, USA...","(25.7283709, -80.238272, 0.0)",25.728371,-80.238272,0.0
4,Saint Regis\r9705 Collins Ave 1404N\rBal Harbour,4/28/2022,8/10/2022,90,"$3,250,000.00",1685,"$1,928.78",Jose Correa Lima,Yaffe International Realty,Sergio Balinotti,"BRG International , LLC",Saint Regis 9705 Collins Ave 1404N Bal Harbour,Saint Regis,9705 Collins Ave 1404N Bal Harbour,"(9705 Collins Ave #1404n, Bal Harbour, FL 3315...","(25.8892229, -80.1228812, 0.0)",25.889223,-80.122881,0.0
5,Majestic Tower\r9601 Collins Ave 1703\rBal Har...,4/19/2022,8/9/2022,50,"$2,800,000.00",2160,"$1,296.30",Sergio Balinotti,"BRG International , LLC",Matias Alem,"BRG International , LLC",Majestic Tower 9601 Collins Ave 1703 Bal Harbour,Majestic Tower,9601 Collins Ave 1703 Bal Harbour,"(9601 Collins Ave APT 1703, Bal Harbour, FL 33...","(25.8876563, -80.1222319, 0.0)",25.887656,-80.122232,0.0
6,Ocean Club\r711 Crandon Blvd 202\rKey Biscayne,7/22/2022,8/10/2022,1,"$2,620,000.00",2080,"$1,259.62",Geysa Guarconi,KB Realty,Geysa Guarconi,KB Realty,Ocean Club 711 Crandon Blvd 202 Key Biscayne,Ocean Club,711 Crandon Blvd 202 Key Biscayne,"(711 Crandon Blvd APT 202, Key Biscayne, FL 33...","(25.6883808, -80.161197, 0.0)",25.688381,-80.161197,0.0
7,The Bond\r1080 Brickell Ave 3500-3501\rMiami,10/12/2021,8/8/2022,303,"$1,910,000.00",3051,$700.40,Lida Juliao,Cervera Real Estate Inc.,Robert Himelfarb,Global Luxury Realty LLC,The Bond 1080 Brickell Ave 3500-3501 Miami,The Bond,1080 Brickell Ave 3500-3501 Miami,"(1080 Brickell Ave 3500 3501, Miami, FL 33131,...","(25.7627976, -80.19265, 0.0)",25.762798,-80.192650,0.0
8,Blue Diamond Condo\r4779 Collins Ave 4002\rMia...,6/12/2022,8/10/2022,25,"$1,810,000.00",1530,"$1,183.01",Claudia Whitechurch,Coldwell Banker Realty,Seth Cohen,Brown Harris Stevens,Blue Diamond Condo 4779 Collins Ave 4002 Miami...,Blue Diamond Condo,4779 Collins Ave 4002 Miami Beach,"(Blue Diamond Tower, 4779 Collins Ave APT 4002...","(25.8232153, -80.121448, 0.0)",25.823215,-80.121448,0.0
9,Solimar\r9559 Collins Ave S7-H\rSurfside,7/11/2022,8/12/2022,8,"$1,800,000.00",1980,$909.09,Edward Ohayon,Jonathan Solomon Realty,Shneur Shapira,The Brokerage S Fl Real Estate,Solimar 9559 Collins Ave S7-H Surfside,Solimar,9559 Collins Ave S7-H Surfside,"(9559 Collins Ave s7 h, Surfside, FL 33154, US...","(25.8862331, -80.1222488, 0.0)",25.886233,-80.122249,0.0
10,T0rump Royale\r18201 Collins Ave 3809\rSunny I...,3/4/2022,8/12/2022,108,"$1,775,000.00",1722,"$1,030.78",Yanina Velez,"Grand Realty of America, Corp.",Milana Nosikovsky PA,Exclusively Baronoff Rlty LLC,T0rump Royale 18201 Collins Ave 3809 Sunny Isl...,Trump Royale,18201 Collins Ave,"(Trump Royale, 18201 Collins Ave #3809, Sunny ...","(25.944

Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

## Correction Section
To be used if a name is off. Format for making corrections is as follows:

`df.at[index_num,'column_name']=('what you want inserted')`

In [14]:
df.at[10,'building_name']=('Trump Royale')
df.at[10,'address_only']=('18201 Collins Ave')

In [16]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=11)
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format('August 7th - August 13th')

# marker_cluster = MarkerCluster().add_to(m)

for i,r in df.iterrows():
    location = (r["lat"], r["lon"])
    info = (r["Sale Price"], r["days_on_market"], r["Agent"], r['Buyer Agent'], r['price_per_sqft'], r['address_only'], r['building_name'])
    info = list(info)
    new_line = '<br>'
    bold_start = '<strong>'
    bold_end = '</strong>'
    text = f'Building Name: {bold_start}{info[-1]}{bold_end}{new_line} \
    Sale Price: {bold_start}{info[0]}{bold_end}{new_line}Days on Market: \
    {bold_start}{info[1]}{bold_end}{new_line}Listing Agent: {bold_start}{info[2]} \
    {bold_end}{new_line}Buying Agent: {bold_start}{info[3]}{bold_end}{new_line}Price sq ft: \
    {bold_start}{info[4]}{bold_end}{new_line}Address: {bold_start}{info[5]}{bold_end}'
    folium.Marker(location=location,
                      tooltip=text)\
    .add_to(m)

m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))

m

In [17]:
m.save('index.html')

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [18]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [19]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/CondoSalesWeekEnding08_13_22
